In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Read the seismic dataset
df = pd.read_excel("/Users/renyijing/Desktop/data/china.xls")
df.head(5)

In [ ]:
# Change the data frame to English for statistics
df.columns = ["id", "date", "lon", "lat", "depth", "type", "level", "loc", "incident"]
print(df.shape, "\n", df.dtypes)
df.head(5)

# Data Cleaning

In [ ]:
# Calculate the number of NaN
x = df.isnull().sum().sum()
print("共有NaN：", x)

# Count the number of duplicate rows
x = df.duplicated().sum()
print("共有重复行：", x)

# Data Wrangling

In [ ]:
# Data processing at the time of the earthquake is accurate to the month and hour
# Convert to time format
df['date'] = pd.to_datetime(df['date'])
# Get the year and month
df['month'] = df['date'].apply(lambda x: str(x)[:7])
# Get the hour
df['hour'] = df['date'].dt.hour
df.head(5)

In [ ]:
# Coordinate conversion, updating the corresponding latitude and longitude column in the DataFrame
import requests

longitude_list = []
latitude_list = []
# The latitude and longitude of Baidu map is converted to the latitude and longitude of AutoNavi map
for i, location in enumerate(df[['lon','lat']].values):
    location = str(location[0])+','+str(location[1])
    url = 'https://restapi.amap.com/v3/assistant/coordinate/convert?'
    parames = {
        'locations':location,
        'coordsys':'baidu',
        'key':'2b7d8c3f14205821d7b0c591a7b7e1fb',
        }
    # Convert a list of strings in JSON data to a list
    r = eval(requests.get(url, params=parames).json()['locations'])
    # longitude
    longitude_list.append(r[0])
    # latitude
    latitude_list.append(r[1])
    print(f'\r{i+1}',end='')
# Add the converted latitude and longitude data to the original DataFrame    
df['longitude'] = longitude_list
df['latitude'] = latitude_list

In [ ]:
# Through the reverse geocoding API of AutoNavi Map, the longitude and latitude information is converted into the corresponding cities and provinces, and the corresponding columns in the DataFrame are updated
citys = []
provinces = []
# Loop through the reverse geocoding
for i, location in enumerate(df[['lon','lat']].values):
    location = str(location[0])+','+str(location[1])
    url = 'https://restapi.amap.com/v3/geocode/regeo?'
    params = {
        'location':location,
        'key':'2b7d8c3f14205821d7b0c591a7b7e1fb',
        'extensions':'base',
        'batch':'false',
        'roadlevel':0,
        }
    r = requests.get(url, params=params)
    data = r.json()['regeocode']
    city = data['addressComponent']['city']
    province = data['addressComponent']['province']
    if len(city)==0:
        city = province
    citys.append(city)
    provinces.append(province)
    print(f'\r{i+1}',end='')
# Add the obtained city and province information to the original DataFrame
df['city'] = citys
df['province'] = provinces

In [ ]:
# Areas that do not have a clear city or province are classified as border areas of other sea areas
df['city'] = df['city'].replace('[]', '其他海域边境地区')
df['city'] = df['city'].replace('中华人民共和国', '其他海域边境地区')
df['province'] = df['province'].replace(' [] ', '其他海域边境地区')
df['province'] = df['province'].replace('中华人民共和国','其他海域边境地区')
print(df.head(5))

# Statistics and Visualization

The number of earthquakes with level>=4.5 in each province from 2013 to 2020

In [ ]:
from wordcloud import WordCloud
from pyecharts.globals import SymbolType

# Convert the list in the province column to a string
df['province'] = df['province'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Filter out rows with province set to []
df_cn = df[df['province'] != '[]']

# Select a row with a seismic magnitude greater than or equal to 4.5
df_severe = df_cn[df_cn['level'] >= 4.5]

# Group by province and count the number of earthquakes in each province
df_province = df_severe.groupby('province')['date'].count().to_frame('次数').sort_values(by='次数', ascending=False).reset_index()

# Generate a word frequency dictionary
wordcloud_data = dict(zip(df_province['province'], df_province['次数']))

# Create a word cloud object
wordcloud = WordCloud(font_path='/System/Library/Fonts/PingFang.ttc',
                      width=800, 
                      height=400, 
                      background_color='white').generate_from_frequencies(wordcloud_data)
# Draw a word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.title('2013-2020年各省份M>=4.5的地震次数', fontsize=15) 
plt.show()

Cloud map of the top 15 cities with the number of earthquakes with M>=4.5 in each province from 2013 to 2020

In [ ]:
# Convert the list in the city column to a string
df['city'] = df['city'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Filter out rows where city is [].
df_cm = df[df['city'] != '[]']

# Select a row with a seismic magnitude greater than or equal to 4.5
df_level = df_cm[df_cm['level'] >= 4.5]

# Group by city and count the number of earthquakes in each city
df_city = df_level.groupby('city')['date'].count().to_frame('次数').sort_values(by='次数', ascending=False).reset_index()
print(df_city.head(14))

# Generate a word frequency dictionary
wordcloud_data1 = dict(zip(df_city['city'], df_city['次数']))

# Create a word cloud object
wordcloud = WordCloud(font_path='/System/Library/Fonts/PingFang.ttc',
                      width=800, 
                      height=400, 
                      background_color='white').generate_from_frequencies(wordcloud_data1)
# Draw a word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.title('2013-2020年发生M>=4.5地震次数前15城市', fontsize=15) 
plt.show()

Heat map of earthquake distribution in China from 2013 to 2020

In [ ]:
import plotly.graph_objects as go

# Filter rows with seismic magnitude greater than or equal to 4.5 and store the results in df_data
df_data = df.query('level >= 4.5')

# A Densitymapbox object is created
my_map = go.Densitymapbox(lat=df_data['lat'], lon=df_data['lon'], z=df_data['level'], radius=10)
fig = go.Figure(my_map)
p1 = fig.update_layout(mapbox_style="open-street-map")


p1.write_html("p1.html", include_plotlyjs='cdn')
p1

Scatter plot of the magnitude and source depth of earthquakes in China from 2013 to 2020

In [ ]:
# Extract magnitude and source depth data
level = df['level']
depth = df['depth']

# Draw a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(level, depth, c='blue', alpha=0.5, s=6)

# Add a title and axis labels
plt.title('2013-2020年震级震源深度散点图')
plt.xlabel('震级（M）')
plt.ylabel('深度 (km)')

# Display graphics
plt.grid(True)
plt.show()

Number of earthquakes in 2013-2020 Nightingale Rose Chart

In [ ]:
df_n = pd.DataFrame(df)
df_n['month'] = pd.to_datetime(df_n['month'])
df_n['year'] = df_n['month'].dt.year

# Count the number of earthquakes by year
yearly_counts = df_n['year'].value_counts().sort_index()
print(yearly_counts)

# Calculate the angle of each sector
total_years = len(yearly_counts)
angles = np.linspace(0, 2 * np.pi, total_years, endpoint=False)

# Create the Nightingale Rose Diagram
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

# Use a histogram to represent the number of earthquakes per year
bars = ax.bar(angles, yearly_counts, align='center', alpha=0.7)

# Set up labels
ax.set_xticks(angles)
ax.set_xticklabels(yearly_counts.index)

plt.title('2013-2020各年分发生的地震数图')
plt.show()

Histogram of the percentage of earthquakes occurring in a 24-hour period

In [ ]:
df_h = pd.DataFrame(df)

# Convert the "date" column to the datetime type
df_h['date'] = pd.to_datetime(df_h['date'])

# Extract hour information
df_h['hour'] = df_h['date'].dt.hour

# Count the number of earthquakes per hour
hourly_counts = df_h['hour'].value_counts().sort_index()

# Draw a histogram
plt.figure(figsize=(10, 5))
hourly_counts.plot(kind='bar', color='skyblue')
        
plt.title('24小时段地震发生次数图')
plt.xlabel('时刻')
plt.ylabel('发生地震次数')
plt.xticks(rotation=0)
plt.show()

Top 10 worst earthquakes of 2013-2020

In [ ]:
df_l = pd.DataFrame(df)

# Sort by level column in descending order
sorted_df = df_l.sort_values(by='level', ascending=False)

# Get the top ten largest items
top_ten_items = sorted_df.head(16)

# Output the result
print(top_ten_items)

The number of earthquakes that occurred in each magnitude range

In [ ]:
df_a = pd.DataFrame(df)

# Define the scope
ranges = [1, 3, 4.5, 6, 7, 8]

# Use the cut method to divide the data into different ranges
df_a['level_range'] = pd.cut(df_a['level'], bins=ranges)

# The number of counts for each range
level_counts = df_a['level_range'].value_counts().sort_index()

# Output the result
print(level_counts)

Statistics for each year

In [ ]:
df_b = pd.DataFrame(df)
df_b['date'] = pd.to_datetime(df_b['date'])  # 将 date 列转换为 datetime 类型

# Year of extraction
df_b['year'] = df_b['date'].dt.year

# Grouped by year, the number of earthquakes, the average magnitude, and the average depth are calculated each year
result = df_b.groupby('year').agg({
    'date': 'count',        # Number of earthquakes
    'level': 'mean',        # Average magnitude
    'depth': 'mean'         # Average depth
}).reset_index()

# Rename the column name
result.columns = ['year', 'earthquake_count', 'average_level', 'average_depth']

# Output the result
print(result)

Collect earthquake depth information for each year

In [ ]:
df_v = pd.DataFrame(df)

# Count the maximum and minimum values of the depth column
max_depth = df_v['depth'].max()
min_depth = df_v['depth'].min()

# Output the result
print("最大深度:", max_depth)
print("最小深度:", min_depth)

# Define the scope
ranges_v = [0,10,20,100,700]

# Use the cut method to divide the data into different ranges
df_v['depth_range'] = pd.cut(df_a['depth'], bins=ranges_v)

# The number of counts for each range
depth_counts = df_v['depth_range'].value_counts().sort_index()

# Output the result
print(depth_counts)